Let's try to make a notebook that looks at the PSF.

In [ ]:
from lsst.daf.persistence import Butler

In [ ]:
butler = Butler('/home/shared/twinkles/output_data_v2')

In [ ]:
subset = butler.subset('calexp', filter='i')

In [ ]:
dataId = subset.cache[6]

In [ ]:
my_calexp = butler.get('calexp', **dataId)

In [ ]:
psf = my_calexp.getPsf()

In [ ]:
shape = psf.computeShape()

In [ ]:
shape.getDeterminantRadius()

In [ ]:
from lsst.afw.geom import Point2D
point = Point2D(50.1, 160.2)

In [ ]:
shape = psf.computeShape(point)

In [ ]:
shape.getDeterminantRadius()

In [ ]:
shapes = []
for x in range(100):
    for y in range(100):
        point = Point2D(x*40., y*40.)
        shapes.append(psf.computeShape(point).getDeterminantRadius())

In [ ]:
from matplotlib import pylab as plt

In [ ]:
plt.hist(shapes)

In [ ]:
src = butler.get('src', **dataId)

In [ ]:
import numpy
star_idx = numpy.where(src.get('base_ClassificationExtendedness_value') == 0)

In [ ]:
import math
star_shapes = []
psf_shapes = []
for x, y in zip(src.getX()[star_idx[0]], src.getY()[star_idx[0]]):
    point = Point2D(x, y)
    psf_shapes.append(psf.computeShape(point).getDeterminantRadius())
for xx, yy in zip(src.get('base_SdssShape_psf_xx')[star_idx[0]], src.get('base_SdssShape_psf_yy')[star_idx[0]]):
    star_shapes.append(math.sqrt(xx + yy))
star_shapes = numpy.array(star_shapes)
psf_shapes = numpy.array(psf_shapes)

In [ ]:
good_star_idx = numpy.where(numpy.isfinite(star_shapes))[0]

In [ ]:
plt.hist(psf_shapes[good_star_idx])

In [ ]:
plt.hist(star_shapes[good_star_idx])

In [ ]:
plt.scatter(star_shapes[good_star_idx], star_shapes[good_star_idx]/psf_shapes[good_star_idx]*(math.sqrt(2)/2), alpha=0.5)
plt.ylim(1.00075, 1.00175)